In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import requests
import re
import time

#Import standard libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
from nltk.tokenize.toktok import ToktokTokenizer
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import spacy
nlp = spacy.load('en_core_web_sm', disable=['ner'])


import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
from bs4 import BeautifulSoup
# import contractions
# contractions.fix("you're happy now")
# from contractions import CONTRACTION_MAP
import unicodedata


In [ ]:
# !python -m spacy download en_core_web_sm

In [ ]:

# headers = {
#     'Referer': 'https://www.rottentomatoes.com/m/notebook/reviews?type=user',
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.108 Safari/537.36',
#     'X-Requested-With': 'XMLHttpRequest',
# }

# s = requests.Session()
        
# def get_reviews(url):
#     r = requests.get(url)
#     movie_id = re.findall(r'(?<=movieId":")(.*)(?=","type)',r.text)[0]

#     api_url = f"https://www.rottentomatoes.com/napi/movie/{movie_id}/reviews/user" #use reviews/userfor user reviews
#     # https://www.rottentomatoes.com/napi/movie/{movie_id}/criticsReviews/all
#     print(api_url)
#     payload = {
#         'direction': 'next',
#         'endCursor': '',
#         'startCursor': '',
#     }
    
#     review_data = []
#     count = 0
#     while True:
#         r = s.get(api_url, headers=headers, params=payload)
#         data = r.json()

#         if not data['pageInfo']['hasNextPage']:
#             break

#         payload['endCursor'] = data['pageInfo']['endCursor']
#         payload['startCursor'] = data['pageInfo']['startCursor'] if data['pageInfo'].get('startCursor') else ''

#         review_data.extend(data['reviews'])
#         time.sleep(0.01)
#         count = count + 1
#         if count > 1000:
#           break

#     return review_data

# data = get_reviews('https://www.rottentomatoes.com/m/top_gun_maverick/reviews?type=user')
# df = pd.json_normalize(data)
# df.to_csv('top_gun_maverick.csv')

In [ ]:
df= pd.read_csv("/content/top_gun_maverick.csv" , index_col=[0])
df.head()

In [ ]:
df.info()

In [ ]:
df1 = df.copy(deep = True)

In [ ]:
len(df1)

In [ ]:
df1 = df1.drop(['displayName','isVerified','isSuperReviewer','hasSpoilers','hasProfanity','rating','timeFromCreation','user.displayName',
         'user.accountLink','user.realm','user.userId','displayImageUrl'],axis = 1)

In [ ]:
df1.head()

In [ ]:
df1['label'] = [0 if x<3 else 1 for x in df1['score']]
df1.head()

In [ ]:
df1 = df1.drop(['score'],axis = 1)

In [ ]:
df1.head()

In [ ]:
df1.isna().sum()

In [ ]:
pie=df1['label'].value_counts()
pie

In [ ]:
#to visualize the above information in the pie chart
plt.figure(figsize=(18,6))
plt.pie(pie,labels=['Positive','Negative'],colors=['red','yellow'])
#set the title name with fontsize
plt.title("The label percentage in TOP-GUN:MAVERICK",fontsize=32)
plt.show()

In [ ]:
nltk.download('stopwords')

In [ ]:
import string
string.punctuation

import nltk
from nltk.corpus import stopwords


In [ ]:
#create the remove the stopwords
stop_words_list=nltk.corpus.stopwords.words('english')
stop_words_list.remove('no')
stop_words_list.remove('not')

In [ ]:
#Creat the function and remove the punctuation
def remove_punctuation(text):
 for punctuation in string.punctuation:
   text=text.replace(punctuation,'')
 return text

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text
    
def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

In [ ]:
df1['cleaned_review'] = df1['review'].replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True)

df1['cleaned_review']=df1['review'].str.encode('ascii','ignore').str.decode('ascii')

df1['cleaned_review']=df1['cleaned_review'].apply(remove_punctuation)

df1['cleaned_review']=df1['cleaned_review'].apply(remove_special_characters)

df1['cleaned_review'] = df1['cleaned_review'].apply(lemmatize_text)

df1['cleaned_review'] = df1['cleaned_review'].apply(remove_stopwords)

In [ ]:

print('-------------------------------------------------')

print('Average count of review per sentence in data is: {0:.0f}.'.format(df1['review'].count().mean()))

print('-------------------------------------------------')

print('Average word length [Uncleaned Data] : {0:.0f}.'.format(np.mean(df1['review'].apply(lambda x: len(x.split())))))

print('--------------------------------------------------------')

print('Average word length [Cleaned review data] : {0:.0f}.'.format(np.mean(df1['cleaned_review'].apply(lambda x: len(x.split())))))

In [ ]:
df1.head()

In [ ]:
nltk.download('averaged_perceptron_tagger')

In [ ]:
# demo for POS tagging for sample news headline
sentence = str(df1.iloc[1].cleaned_review)
sentence_nlp = nlp(sentence)

# POS tagging with Spacy 
spacy_pos_tagged = [(word, word.tag_, word.pos_) for word in sentence_nlp]
pd.DataFrame(spacy_pos_tagged, columns=['Word', 'POS tag', 'Tag type'])

# POS tagging with nltk
# nltk_pos_tagged = nltk.pos_tag(sentence.split())
# pd.DataFrame(nltk_pos_tagged, columns=['Word', 'POS tag'])

In [ ]:
from textblob import TextBlob

In [ ]:
# compute sentiment scores (polarity) and labels
df1['sentiment_scores']= [round(TextBlob(reviews).sentiment.polarity, 3) for reviews in df1['cleaned_review']]

In [ ]:
df1.head()

In [ ]:
df1['sentiment_category'] = ['positive' if score > 0 else 'negative' for score in df1['sentiment_scores']]

In [ ]:
df1.head()

In [ ]:
df1.groupby(by=['sentiment_category']).describe()

In [ ]:
df1.head()

In [ ]:
from wordcloud import WordCloud

# mpl.rcParams['font.size']=12     # font size            
# mpl.rcParams['savefig.dpi']=100          
# mpl.rcParams['figure.subplot.bottom']=.1 


def show_wordcloud(data , title = None):
    wordcloud = WordCloud(
        background_color='black', # background color is set to black
        max_words=300, # maximum number of words
        max_font_size=40, # maximum font size is 40
        scale=3,
        random_state=1 # chosen at random by flipping a coin; it was heads
        
    ).generate(str(data))
    
    fig = plt.figure(1, figsize=(15, 15))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()
    
show_wordcloud(df1["cleaned_review"])

In [ ]:
df1[df1['label']== 0].tail(15)

In [ ]:
corpus = df1['cleaned_review'].values
y= df1['label'].values

In [ ]:
#Covert the text into araay using the TfidfVectorizer because the our machine
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#install the model
vector=TfidfVectorizer(lowercase=False)
vector.fit(corpus)
#Transform the above fit the data using the TfidfVectorizer and finally print
X=vector.fit_transform(corpus)
X

In [ ]:
#divided the data for train and test
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state = 123)

In [ ]:
print('Rows in train data:' , X_train.shape[0])
print('Dimensions in train data:' , X_train.shape[1])
print('Rows in Test data:' , X_test.shape[0])
print('Dimesnions in Test data:' , X_test.shape[1])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
models = []

models.append(('Logistic Regression',LogisticRegression()))
models.append(('Random Forest Classifier',RandomForestClassifier()))
models.append(('Decision Tree Classifier',DecisionTreeClassifier()))

In [ ]:
# Data Engineer - Takes care of the data which data scienstst / Data Analyst Needed - Inside SQL (Bunch of quries) - will provide you the required columns

##--> Based on these columns ---> You perform your analysis ---> Data Integrity (DeepCheck) --> What is happening in your data ---> Missing Values, Duplicate Values , Anomalies --> Columns Abusurd Values
# --- > Your data should make sense ---> Dig Deep Into Categorical Columns and Numerical Columns(Descrete) --- > Be Ready detailed Analysis on DATA , Good EDA , 

## ML Engineer --> They know things end-to-end ---> They have expertese in deployment on Cloud Services ---> How to create a CI/CD pipeline

# Excellency in python  XXXX ---> Try to understand and try write your own codes ---> Python

In [ ]:
for name, model in models:

    %time model.fit(X_train, y_train)
    test_pred = model.predict(X_test)
    print(name ,'Accuracy Score : ',accuracy_score(y_test, test_pred))
    print(name ,'F1 Score : ',f1_score(y_test, test_pred, average='weighted'))
    print('-----------------------------------------------------------------------')

## Topic Modelling


In [ ]:
import sys
# !{sys.executable} -m spacy download en
import re, numpy as np, pandas as pd
from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

In [ ]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)

In [ ]:
# Convert to list
data = df1.review.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1])

#### Build the Bigram, Trigram Models and Lemmatize

In [ ]:
 # Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# !python3 -m spacy download en  # run in terminal once
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load("en_core_web_sm")
    #nlp = spacy.load('en', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

data_ready = process_words(data_words)  # processed Text Data!

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_ready)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

pprint(lda_model.print_topics())

In [ ]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [ ]:
df_topic = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_ready)

In [ ]:
df_topic.head()

In [ ]:
df_dominant_topic = df_topic.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(5)

In [ ]:
# Display setting to show more characters in column
pd.options.display.max_colwidth = 100

sent_topics_sorteddf_mallet = pd.DataFrame()
sent_topics_outdf_grpd = df_topic.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

# Show
sent_topics_sorteddf_mallet.head(10)

In [ ]:
# 1. Wordcloud of Top N words in each topic
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'

cloud = WordCloud(stopwords=stop_words,
                  background_color='white',
                  width=2500,
                  height=1800,
                  max_words=10,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = lda_model.show_topics(formatted=False)

fig, axes = plt.subplots(2, 2, figsize=(10,10), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=300)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')


plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show()

In [ ]:
from collections import Counter
topics = lda_model.show_topics(formatted=False)
data_flat = [w for w_list in data_ready for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])        

# Plot Word Count and Weights of Topic Keywords
fig, axes = plt.subplots(2, 2, figsize=(16,10), sharey=True, dpi=160)
cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]
for i, ax in enumerate(axes.flatten()):
    ax.bar(x='word', height="word_count", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.5, alpha=0.3, label='Word Count')
    ax_twin = ax.twinx()
    ax_twin.bar(x='word', height="importance", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.2, label='Weights')
    ax.set_ylabel('Word Count', color=cols[i])
    ax_twin.set_ylim(0, 0.030); ax.set_ylim(0, 3500)
    ax.set_title('Topic: ' + str(i), color=cols[i], fontsize=16)
    ax.tick_params(axis='y', left=False)
    ax.set_xticklabels(df.loc[df.topic_id==i, 'word'], rotation=30, horizontalalignment= 'right')
    ax.legend(loc='upper left'); ax_twin.legend(loc='upper right')

fig.tight_layout(w_pad=2)    
fig.suptitle('Word Count and Importance of Topic Keywords', fontsize=22, y=1.05)    
plt.show()

In [ ]:
# !pip install pyLDAvis

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis